In [2]:
# Libraries
import time
import pandas as pd    
# ------------- # 
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
# ------------- # 
from webdriver_manager.chrome import ChromeDriverManager

# library for interacting with the operating system
import os
# ------------- # 
from bs4 import BeautifulSoup
import requests
# ------------- # 
from langdetect import detect
# ------------- # 
import random

 # ------------- # 
from datetime import date

In [3]:
# installing driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [4]:
# Enter to the site

#DATA JUNIOR - AMSTERDAM - PAST 7 DAYS
url='https://nl.indeed.com/jobs?q=data+junior&l=Amsterdam&fromage=7&lang=en&vjk=0a92e6a47795380f'
driver.get(url)
time.sleep(2)

In [5]:
# Accept cookies
driver.find_element(By.XPATH,'''//*[@id="onetrust-reject-all-handler"]''').click()
time.sleep(1)

In [6]:
# go to the end of the page in order for all the elements to be loaded

#job_viewer = driver.find_element(By.XPATH,'''//*[@id="onetrust-reject-all-handler"]''').click()
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [7]:
## GET ALL THE LINKS

links = []

next_page= driver.find_elements(By.CSS_SELECTOR,"a[data-testid^='pagination-page-next']")
print(next_page)
while len(next_page)>0:
    time.sleep(random.randint(1,4))
    #jobs_block = driver.find_element(By.XPATH,'//*[@id="mosaic-provider-jobcards"]/ul')
    jobs_list= driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')
    #print("jobs_block",jobs_block)
    print(len(jobs_list))

    for job in jobs_list:
        #all_links = job.find_elements(By.TAG_NAME,'a')
        all_links= driver.find_elements(By.CSS_SELECTOR,"a[class^='jcs-JobTitle css-jspxzf eu4oa1w0']")
        #print(len(all_links))
        for a in all_links:
            if a.get_attribute('href') not in links: 
                links.append(a.get_attribute('href'))

                
        #     else:
        #         pass
        # # scroll down for each job element
        # driver.execute_script("arguments[0].scrollIntoView();", job)

    #print(f'Collecting the links in the page: {page-1}')
    # # go to next page:
    # print('going to next page')
    # driver.find_element(By.XPATH,f"//button[@aria-label='Page {page}']").click()
    # time.sleep(random.randint(1,4))
    next_page[0].click()
    print('hey')
    time.sleep(random.randint(1,4))
    next_page= driver.find_elements(By.CSS_SELECTOR,"a[data-testid^='pagination-page-next']")
else:

    time.sleep(random.randint(1,4))
    #jobs_block = driver.find_element(By.XPATH,'//*[@id="mosaic-provider-jobcards"]/ul')
    jobs_list= driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')
    #print("jobs_block",jobs_block)
    print(len(jobs_list))

    for job in jobs_list:
        #all_links = job.find_elements(By.TAG_NAME,'a')
        all_links= driver.find_elements(By.CSS_SELECTOR,"a[class^='jcs-JobTitle css-jspxzf eu4oa1w0']")
        #print(len(all_links))
        for a in all_links:
            if a.get_attribute('href') not in links: 
                links.append(a.get_attribute('href'))

print('Found ' + str(len(links)) + ' links for job offers')

[<selenium.webdriver.remote.webelement.WebElement (session="e1b0aacea64137a6506ab72bb6f473c8", element="4ee4c216-ae8f-4e1f-8ab4-ec629844c511")>]
15
hey
15
hey
15
hey
6
Found 51 links for job offers


In [8]:
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []


In [9]:
#return do the initial page
time.sleep(2)
driver.get(url)

In [10]:
##GET THE INFP FROM ALL THE PAGES

#FOR THE INITIAL PAGE
#

## get the titles, company and location
all_titles_page=driver.find_elements(By.CSS_SELECTOR, 'span[title]')
#titles=[title.text for title in all_titles_page]
for title in all_titles_page:
    job_titles.append(title.text)

all_companies_page=driver.find_elements(By.CSS_SELECTOR, 'span[class^=companyName]')
#company=[comp.text for comp in all_companies_page]
for comp in all_companies_page:
    company_names.append(comp.text)

all_locations_page=driver.find_elements(By.CSS_SELECTOR, 'div[class^=companyLocation]')
#location=[loc.text for loc in all_locations_page]
for loc in all_locations_page:
    company_locations.append(loc.text)


## CHECKIN IF THERE ARE MORE PAGES AND GOING THERE

next_page= driver.find_elements(By.CSS_SELECTOR,"a[data-testid^='pagination-page-next']")
while len(next_page)>0:
    next_page[0].click()
    #print('hey')
    time.sleep(random.randint(1,4))
    ## get the titles, company and location
    all_titles_page=driver.find_elements(By.CSS_SELECTOR, 'span[title]')
    #titles=[title.text for title in all_titles_page]
    for title in all_titles_page:
        job_titles.append(title.text)

    all_companies_page=driver.find_elements(By.CSS_SELECTOR, 'span[class^=companyName]')
    #company=[comp.text for comp in all_companies_page]
    for comp in all_companies_page:
        company_names.append(comp.text)

    all_locations_page=driver.find_elements(By.CSS_SELECTOR, 'div[class^=companyLocation]')
    #location=[loc.text for loc in all_locations_page]
    for loc in all_locations_page:
        company_locations.append(loc.text)
    time.sleep(random.randint(1,4))

    next_page= driver.find_elements(By.CSS_SELECTOR,"a[data-testid^='pagination-page-next']")

In [11]:
job_descp=["n/a" for title in job_titles]
job_lan=["en" for title in job_titles]

In [12]:
len(job_titles)

51

In [13]:
len(company_names)

51

In [14]:
len(company_locations)

51

In [15]:
# Creating the dataframe 
df = pd.DataFrame(list(zip(job_titles,company_names,
                    company_locations,job_descp,job_lan,links)),
                    columns =['job_title', 'company_name',
                           'company_location','job_desc','lan','link'])
df

,job_title,company_name,company_location,job_desc,lan,link
0,Personal Trainer,Ultimate Performance,1016 Amsterdam Centrum\n+1 plaats,n/a,en,https://nl.indeed.com/pagead/clk?mo=r&ad=-6NYl...
1,(Junior) Procurement Specialist Rotterdam (m/w/d),Coolblue,Amsterdam\n+1 plaats,n/a,en,https://nl.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,"Regional Operations Manager, Uber Direct",Uber,Tijdelijk thuiswerken in 1097 Amsterdam,n/a,en,https://nl.indeed.com/pagead/clk?mo=r&ad=-6NYl...
3,(Junior) Category Manager Rotterdam (m/w/d),Coolblue,Amsterdam\n+1 plaats,n/a,en,https://nl.indeed.com/pagead/clk?mo=r&ad=-6NYl...
4,Junior Data Engineer,Independent Recruiters,Amsterdam\n+1 plaats,n/a,en,https://nl.indeed.com/pagead/clk?mo=r&ad=-6NYl...
5,Jr. Data Engineer KPN,Harvey Nash BV,Hybride werken in 1043 Amsterdam Westpoort,n/a,en,https://nl.indeed.com/company/Harvey-Nash-BV/j...
6,Junior Desktop Support Analyst,Arup,Amsterdam,n/a,en,https://nl.indeed.com/rc/clk?jk=816e86f245387f...
7,6180 Junior Data Engineer Bios CJ,Magno IT Recruitment,Amsterdam,n/a,en,https://nl.indeed.com/rc/clk?jk=7cf7d64da785ad...
8,Scientist/Junior Scientist,Sotio,1077 Amsterdam Zuid,n/a,en,https://nl.indeed.com/rc/clk?jk=feca0239ea9292...
9,Junior Data Analyst,Levy Associates Ltd,Amsterdam,n/a,en,https://nl.indeed.com/rc/clk?jk=0a92e6a4779538...


In [16]:
# EXPORT DATAFRAME TO EXCEL
date=date.today()
filename=f"Excel_files/Indeed_juniordata_{date}.xlsx"
df.to_excel(filename,index=False)